In [ ]:
# Standard library
import os

# Third-party
import astropy.time as atime
from astropy import log as logger
import astropy.units as u
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# Project
from thejoker import Paths
paths = Paths()
from thejoker.data import RVData
from thejoker.units import usys
from thejoker.util import quantity_from_hdf5
from thejoker.celestialmechanics import OrbitalParams, SimulatedRVOrbit, rv_from_elements
from thejoker.plot import plot_rv_curves, plot_corner

plt.style.use('../thejoker/thejoker.mplstyle')

In [ ]:
samples_filename = "../cache/2M00110648+6609349.h5"
data_filename = "../data/troup-allVisit.h5"
plot_path = "../paper/figures/"
os.makedirs(plot_path, exist_ok=True)

## First read the orbital parameters sampled for this experiment:

In [ ]:
name = os.path.splitext(os.path.split(samples_filename)[1])[0]
hdf5_key = name
print(name)

In [ ]:
with h5py.File(samples_filename, 'r') as g:
    jitter = g.attrs['jitter_m/s']*u.m/u.s
    P_min = g.attrs['P_min_day']*u.day
    P_max = g.attrs['P_max_day']*u.day

    # read the orbital parameters
    opars = OrbitalParams.from_hdf5(g)

## read the data:

In [ ]:
with h5py.File(data_filename, 'r') as f:
    g = f[hdf5_key]

    bmjd = g['mjd'][:]
    rv = quantity_from_hdf5(g, 'rv')
    rv_err = quantity_from_hdf5(g, 'rv_err')
data = RVData(bmjd, rv, stddev=rv_err)

### Read in Troup's fit values for the orbital parameters:

In [ ]:
all_troup_data = np.genfromtxt(os.path.join(paths.root, "data", "troup16-dr12.csv"),
                               delimiter=",", names=True, dtype=None)
troup_data = all_troup_data[all_troup_data['APOGEE_ID'].astype(str) == name]

In [ ]:
troup_P = troup_data['PERIOD'] * u.day
troup_ecc = troup_data['ECC']
_,troup_asini = SimulatedRVOrbit.P_K_ecc_to_mf_asini_ecc(troup_P, troup_data['SEMIAMP']*u.m/u.s, troup_ecc)
troup_omega = (troup_data['OMEGA'] % (2*np.pi)) * u.radian

dt0 = atime.Time(troup_data['TPERI'], format='jd', scale='tcb').tcb.mjd - data.t_offset
troup_phi0 = 2*np.pi*dt0/troup_P.value * u.radian

troup_v0 = troup_data['V0'] * u.m/u.s
print(troup_data['SLOPE'])

In [ ]:
troup_truths = np.concatenate((np.log(troup_P.to(u.day).value),
                               np.log(troup_asini.to(u.R_sun).value),
                               troup_ecc,
                               troup_omega.to(u.degree).value % 360.,
                               troup_phi0.to(u.degree).value % 360.,
                               troup_v0.to(u.km/u.s).value))

## RV curves

In [ ]:
troup_orbit = SimulatedRVOrbit(P=troup_P, a_sin_i=troup_asini, ecc=troup_ecc, 
                               omega=troup_omega, phi0=troup_phi0, v0=troup_v0)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))

data.plot(ax=ax, ecolor='k', markersize=3, elinewidth=2, alpha=1, rv_unit=u.km/u.s)

t_grid = np.linspace(data._t.min()-100, data._t.max()+100, 1024) + data.t_offset
model_rv = troup_orbit.generate_rv_curve(t_grid)

ax.plot(t_grid, model_rv, marker=None, alpha=0.5, color='r')

In [ ]:
def opt_func(p):
    _omega, _phi0, _v0 = p
    _orbit = SimulatedRVOrbit(P=troup_P, a_sin_i=-troup_asini, ecc=troup_ecc, 
                              omega=_omega*usys['angle'], phi0=_phi0*usys['angle'], v0=_v0*usys['speed'])
    
    model_rv = _orbit.generate_rv_curve(data._t)
    return (data._rv - model_rv.decompose(usys).value) * np.sqrt(data._ivar)

In [ ]:
from scipy.optimize import leastsq

In [ ]:
p0 = [troup_omega.decompose(usys).value, troup_phi0.decompose(usys).value, troup_v0.decompose(usys).value]
p_opt,ier = leastsq(opt_func, p0)
print(ier)
print(p0, p_opt)

In [ ]:
opt_troup_orbit = SimulatedRVOrbit(P=troup_P, a_sin_i=-troup_asini, ecc=troup_ecc, 
                                   omega=p_opt[0]*usys['angle'], phi0=p_opt[1]*usys['angle'], 
                                   v0=p_opt[2]*usys['speed'])

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))

data.plot(ax=ax, ecolor='k', markersize=3, elinewidth=2, alpha=1, rv_unit=u.km/u.s)

t_grid = np.linspace(data._t.min()-100, data._t.max()+100, 1024) + data.t_offset
model_rv = opt_troup_orbit.generate_rv_curve(t_grid)

ax.plot(t_grid, model_rv, marker=None, alpha=0.5, color='r')

---

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))

dmjd = bmjd.max() - bmjd.min()
t_grid = np.linspace(bmjd.min() -0.25*dmjd, 
                     bmjd.max() + 0.25*dmjd, 
                     1024)

rv_unit = u.km/u.s

plot_rv_curves(opars[:512], t_grid, rv_unit=rv_unit,
               data=data, ax=ax, 
               plot_kwargs={'color': '#888888', 'zorder': -100},
               data_plot_kwargs={'ecolor': 'k', 'markersize': 3, 'elinewidth': 1, 'alpha': 1., 'zorder': 100})

ax.set_rasterization_zorder(-1)
ax.set_ylim(-40, -18)

ax.set_title(name)
fig.tight_layout()
fig.savefig(os.path.join(plot_path, 'exp1-rv-curves.pdf'), dpi=100) # change to: dpi=256 for production?

## Corner

In [ ]:
samples = opars.pack(plot_units=True)

_med_v0 = np.median(samples[:,5])
_mad_v0 = np.median(np.abs(samples[:,5] - _med_v0))

ranges = [
    (4.5, 7.5),
    (1.5, 5.5),
    (0,1),
    (0,360),
    (0,360),
    (_med_v0 - 5*_mad_v0, _med_v0 + 5*_mad_v0)
]

In [ ]:
fig = plot_corner(opars, alpha=0.02, truths=troup_truths, range=ranges, 
                  plot_contours=True, plot_density=True, bins=32)
fig.suptitle(name, fontsize=26)
fig.savefig(os.path.join(plot_path, 'exp1-corner.pdf'), dpi=100) # change to: dpi=256 for production?